In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


C:\Users\utilisateur\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning:

Columns (0,1,18) have mixed types.Specify dtype option on import or set low_memory=False.



,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
198,AFG,Asia,Afghanistan,2020-07-26,36157.0,121.0,1259.0,13.0,928.809,3.108,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
338,ALB,Europe,Albania,2020-07-26,4637.0,67.0,134.0,6.0,1611.300,23.282,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
542,DZA,Africa,Algeria,2020-07-26,26764.0,605.0,1146.0,10.0,610.339,13.797,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-25,France,181534.0,0.0,30209.0,0.0,0.0,0.0
0,2020-07-25,China,86333.0,0.0,4658.0,0.0,0.0,0.0
0,2020-07-25,Italy,246001.0,0.0,35130.0,0.0,0.0,0.0
0,2020-07-25,Spain,274751.0,0.0,28445.0,0.0,0.0,0.0
0,2020-07-25,United States,4175080.0,0.0,146618.0,0.0,0.0,0.0
0,2020-07-25,World,16016494.0,0.0,647445.0,0.0,0.0,0.0
0,2020-07-25,United Kingdom,298934.0,0.0,45779.0,0.0,0.0,0.0
0,2020-07-25,Germany,205768.0,0.0,9135.0,0.0,0.0,0.0
0,2020-07-25,Iran,288593.0,0.0,15321.0,0.0,0.0,0.0
0,2020-07-25,Turkey,225127.0,0.0,5597.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-07-26,France,181534.0,180528.0,30209.0,30192.0,1006.0,17.0
0,2020-07-26,China,86333.0,86381.0,4658.0,4652.0,-48.0,6.0
0,2020-07-26,Italy,246001.0,245864.0,35130.0,35102.0,137.0,28.0
0,2020-07-25,Spain,274751.0,272421.0,28445.0,28432.0,2330.0,13.0
0,2020-07-26,United States,4175080.0,4178027.0,146618.0,146460.0,-2947.0,158.0
0,2020-07-26,World,16016494.0,16018105.0,647445.0,644832.0,-1611.0,2613.0
0,2020-07-26,United Kingdom,298934.0,298681.0,45779.0,45738.0,253.0,41.0
0,2020-07-26,Germany,205768.0,205269.0,9135.0,9118.0,499.0,17.0
0,2020-07-26,Iran,288593.0,284034.0,15321.0,15074.0,4559.0,247.0
0,2020-07-26,Turkey,225127.0,225173.0,5597.0,5596.0,-46.0,1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")